In [2]:
import os
import sys
import pandas as pd
import numpy as np
import PIL
import matplotlib.pyplot as plt

%matplotlib inline

seed = 16
np.random.seed(seed)

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#check using system GPU for processing and declaring system/GPU parameters

from tensorflow.python.client import device_lib
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu
print(device_lib.list_local_devices())

# configure tensorflow before fitting model
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.Session(config=tf_config)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15200755127104026535
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1493781708
locality {
  bus_id: 1
}
incarnation: 1268612039573125787
physical_device_desc: "device: 0, name: GeForce GTX 860M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [4]:
# changing directory for flow_from_directory method
os.chdir('C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets')

In [22]:
num_classes = 120
batch_size = 10

train_datagen = ImageDataGenerator(rotation_range=15, shear_range=0.1, channel_shift_range=20,
                                    width_shift_range=0.1,  height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True,
                                    fill_mode='nearest', rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory('train', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)

validation_generator = validation_datagen.flow_from_directory('validation', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)


test_generator = test_datagen.flow_from_directory('test', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)


Found 12000 images belonging to 120 classes.
Found 2400 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [21]:
input_shape = (224, 224, 3)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import keras.utils
from keras.applications.imagenet_utils import decode_predictions
from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='saved_models/weights.bestaugmented.pre_trained.hdf5', 
                               verbose=1, save_best_only=True)

In [7]:
from keras.applications import VGG16
# Load the VGG model
# choosing VG166 architecture initially as my hand-built models with 3x3 filters perform the best and generally is more popular

vgg16_base = Sequential()
vgg16_base.add(VGG16(weights='imagenet',
                  include_top=False,
                pooling='avg'))
vgg16_base.add(Dense(num_classes, activation='softmax'))
vgg16_base.layers[0].trainable = False

# compile
adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
vgg16_base.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(vgg16_base.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 120)               61560     
Total params: 14,776,248
Trainable params: 61,560
Non-trainable params: 14,714,688
_________________________________________________________________
None


In [10]:
vgg16_base.fit_generator(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=800, 
                    epochs=10, 
                    validation_steps=200,
                    callbacks=[early_stopping, tensorboard])

Epoch 1/10
800/800 [==============================] - 382s 478ms/step - loss: 5.0060 - acc: 0.0078 - val_loss: 4.9803 - val_acc: 0.0092
Epoch 2/10
800/800 [==============================] - 380s 475ms/step - loss: 4.9684 - acc: 0.0075 - val_loss: 4.9534 - val_acc: 0.0092
Epoch 3/10
800/800 [==============================] - 382s 477ms/step - loss: 4.9437 - acc: 0.0086 - val_loss: 4.9312 - val_acc: 0.0104
Epoch 4/10
800/800 [==============================] - 379s 474ms/step - loss: 4.9236 - acc: 0.0078 - val_loss: 4.9120 - val_acc: 0.0108
Epoch 5/10
800/800 [==============================] - 385s 481ms/step - loss: 4.9071 - acc: 0.0082 - val_loss: 4.8955 - val_acc: 0.0117
Epoch 6/10
800/800 [==============================] - 379s 474ms/step - loss: 4.8909 - acc: 0.0089 - val_loss: 4.8810 - val_acc: 0.0121
Epoch 7/10
800/800 [==============================] - 379s 474ms/step - loss: 4.8763 - acc: 0.0081 - val_loss: 4.8680 - val_acc: 0.0113
Epoch 8/10
800/800 [============================

In [11]:
vgg16_base.save('vgg16_base.h5')

In [16]:
base_scores = vgg16_base.evaluate_generator(test_generator, steps=25, max_queue_size=10)
print("Accuracy: %.2f%%" % (base_scores[1]*100))

Accuracy: 0.00%


In [20]:
from keras.models import Model
from keras.layers.normalization import BatchNormalization

In [9]:
base_model = VGG16(include_top=False, input_shape=input_shape)
from keras.layers import GlobalAveragePooling2D

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)

# let's add a fully-connected layer

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)


predictions = Dense(num_classes, activation='softmax')(x)


model = Model(input=base_model.input, output=predictions)

# train only the top layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [11]:
model.fit_generator(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=800, 
                    epochs=10, 
                    callbacks=[early_stopping])

Epoch 1/10
800/800 [==============================] - 337s 421ms/step - loss: 5.1689 - acc: 0.0075 - val_loss: 4.8430 - val_acc: 0.0108
Epoch 2/10
800/800 [==============================] - 335s 419ms/step - loss: 5.0327 - acc: 0.0075 - val_loss: 4.8252 - val_acc: 0.0104
Epoch 3/10
800/800 [==============================] - 339s 424ms/step - loss: 4.9732 - acc: 0.0079 - val_loss: 4.8169 - val_acc: 0.0092
Epoch 4/10
800/800 [==============================] - 336s 420ms/step - loss: 4.9408 - acc: 0.0095 - val_loss: 4.8082 - val_acc: 0.0117
Epoch 5/10
800/800 [==============================] - 336s 420ms/step - loss: 4.9290 - acc: 0.0074 - val_loss: 4.8038 - val_acc: 0.0088
Epoch 6/10
800/800 [==============================] - 338s 422ms/step - loss: 4.9110 - acc: 0.0096 - val_loss: 4.7982 - val_acc: 0.0138
Epoch 7/10
800/800 [==============================] - 336s 421ms/step - loss: 4.8971 - acc: 0.0080 - val_loss: 4.7976 - val_acc: 0.0108
Epoch 8/10
800/800 [============================

In [9]:
# define functions to load data

def load_array(fname):
    return np.load(open(fname,'rb'))

In [10]:
# load in labels and data (as tensors)

train_labels=load_array('train_labels.npy')
valid_labels=load_array('valid_labels.npy')

In [11]:
train_tensor=load_array('train_dataset.npy')

In [12]:
def Normalize_Input(X):
    minimum=0
    maximum=255
    X-minimum/(maximum-minimum)
    return X  

In [13]:
train_tensor=Normalize_Input(train_tensor)

In [14]:
valid_tensor=load_array('valid_dataset.npy')

In [15]:
valid_tensor=Normalize_Input(valid_tensor)

In [32]:
num_classes = 120
batch_size = 12
input_shape = (224, 224, 3)

train_datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1)

validation_datagen = ImageDataGenerator()

# note to self... perhaps the imagedatagenerator parameters I had before were root cause of low accuracy...

train_generator = train_datagen.flow(x=train_tensor, y=train_labels, batch_size=batch_size, shuffle=False, seed=16)
validation_generator = validation_datagen.flow(x=valid_tensor, y=valid_labels, batch_size=batch_size, shuffle=False, seed=16)

In [34]:
vgg16_v2 = Sequential()
vgg16_v2.add(VGG16(weights='imagenet',
                  include_top=False,
                pooling='avg'))
vgg16_v2.add(Dense(num_classes, activation='softmax'))
vgg16_v2.layers[0].trainable = False

# compile
vgg16_v2.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
print(vgg16_v2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_2 (Dense)              (None, 120)               61560     
Total params: 14,776,248
Trainable params: 61,560
Non-trainable params: 14,714,688
_________________________________________________________________
None


In [35]:
vgg16_v2.fit_generator(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=800, 
                    epochs=10, 
                    callbacks=[checkpointer, early_stopping])

Epoch 1/10
800/800 [==============================] - 386s 483ms/step - loss: 13.9315 - acc: 0.0296 - val_loss: 13.1307 - val_acc: 0.0708

Epoch 00001: val_loss improved from inf to 13.13068, saving model to saved_models/weights.bestaugmented.pre_trained.hdf5
Epoch 2/10
800/800 [==============================] - 386s 482ms/step - loss: 11.3126 - acc: 0.1309 - val_loss: 9.9514 - val_acc: 0.2108

Epoch 00002: val_loss improved from 13.13068 to 9.95136, saving model to saved_models/weights.bestaugmented.pre_trained.hdf5
Epoch 3/10
800/800 [==============================] - 387s 484ms/step - loss: 9.6743 - acc: 0.2218 - val_loss: 9.1440 - val_acc: 0.2646

Epoch 00003: val_loss improved from 9.95136 to 9.14405, saving model to saved_models/weights.bestaugmented.pre_trained.hdf5
Epoch 4/10
800/800 [==============================] - 384s 480ms/step - loss: 8.4376 - acc: 0.3030 - val_loss: 8.3636 - val_acc: 0.3133

Epoch 00004: val_loss improved from 9.14405 to 8.36364, saving model to saved_m

In [38]:
# another round of training

vgg16_v2.fit_generator(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=800, 
                    epochs=10, 
                    callbacks=[checkpointer, early_stopping])

Epoch 1/10
800/800 [==============================] - 385s 482ms/step - loss: 5.0574 - acc: 0.5710 - val_loss: 6.5399 - val_acc: 0.4671

Epoch 00001: val_loss improved from 6.56802 to 6.53987, saving model to saved_models/weights.bestaugmented.pre_trained.hdf5
Epoch 2/10
800/800 [==============================] - 384s 480ms/step - loss: 5.0060 - acc: 0.5788 - val_loss: 6.7529 - val_acc: 0.4517

Epoch 00002: val_loss did not improve
Epoch 3/10
800/800 [==============================] - 385s 482ms/step - loss: 4.8536 - acc: 0.5948 - val_loss: 6.8956 - val_acc: 0.4446

Epoch 00003: val_loss did not improve


In [39]:
# another round of training, increase batch size

batch_size=20
vgg16_v2.fit_generator(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=480, 
                    epochs=10, 
                    callbacks=[checkpointer, early_stopping])

Epoch 1/10
480/480 [==============================] - 262s 545ms/step - loss: 4.8157 - acc: 0.6049 - val_loss: 6.5087 - val_acc: 0.4596

Epoch 00001: val_loss improved from 6.53987 to 6.50873, saving model to saved_models/weights.bestaugmented.pre_trained.hdf5
Epoch 2/10
480/480 [==============================] - 263s 548ms/step - loss: 4.7754 - acc: 0.5866 - val_loss: 6.5720 - val_acc: 0.4642

Epoch 00002: val_loss did not improve
Epoch 3/10
480/480 [==============================] - 263s 547ms/step - loss: 4.8430 - acc: 0.5951 - val_loss: 6.4419 - val_acc: 0.4683

Epoch 00003: val_loss improved from 6.50873 to 6.44187, saving model to saved_models/weights.bestaugmented.pre_trained.hdf5
Epoch 4/10
480/480 [==============================] - 261s 544ms/step - loss: 4.4866 - acc: 0.6300 - val_loss: 6.6533 - val_acc: 0.4608

Epoch 00004: val_loss did not improve
Epoch 5/10
480/480 [==============================] - 263s 548ms/step - loss: 4.9115 - acc: 0.5936 - val_loss: 6.8625 - val_acc:

In [40]:
# another round of training, maintain batch size

batch_size=20
vgg16_v2.fit_generator(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=480, 
                    epochs=10, 
                    callbacks=[checkpointer, early_stopping])

Epoch 1/10
480/480 [==============================] - 264s 549ms/step - loss: 4.6897 - acc: 0.6092 - val_loss: 6.8537 - val_acc: 0.4517

Epoch 00001: val_loss did not improve
Epoch 2/10
480/480 [==============================] - 262s 547ms/step - loss: 4.6515 - acc: 0.6104 - val_loss: 6.5844 - val_acc: 0.4763

Epoch 00002: val_loss did not improve
Epoch 3/10
480/480 [==============================] - 261s 545ms/step - loss: 4.6084 - acc: 0.6220 - val_loss: 6.5401 - val_acc: 0.4712

Epoch 00003: val_loss did not improve
Epoch 4/10
480/480 [==============================] - 262s 545ms/step - loss: 4.4628 - acc: 0.6340 - val_loss: 6.6814 - val_acc: 0.4667

Epoch 00004: val_loss did not improve
Epoch 5/10
480/480 [==============================] - 263s 548ms/step - loss: 4.6663 - acc: 0.6248 - val_loss: 6.5071 - val_acc: 0.4838

Epoch 00005: val_loss did not improve
Epoch 6/10
480/480 [==============================] - 262s 545ms/step - loss: 4.6263 - acc: 0.6293 - val_loss: 6.8582 - val_a

In [41]:
# now seems that model is overfitting to the training data.  No additional fitting is needed.
vgg16_v2.save('saved_models/vgg16_v2.h5')